# Overview

We need to refine the way users specify colors, to simplify the public API and eliminate inconsistencies.  The challenge is to handle all of the ways in which a user might want to specify per-series / per-datum colors, while avoiding potential ambiguities.

**Use-cases**

* Specify one explicit color for everything.
        toyplot.plot(x, y, color="red")
* Specify an explicit color per series.
        toyplot.plot(x, y, color=["red", "cinnamon", "cardinal"])
* Specify a palette for the series.
        toyplot.plot(x, y, color=toyplot.color.brewer("reds"))
* Specify a colormap for the series.
        toyplot.plot(x, y, color=toyplot.color.LinearMap(toyplot.color.Palette(["red", "green", "blue"])))
* Specify per-datum scalar values and have them mapped.
        toyplot.plot(x, y, color=temperature)
* Specify per-datum scalar values and have them mapped, but override a few "special" values.
        colors = toyplot.color.broadcast(temperature)
        colors[numpy.argsort(temperature)[0]] = "blue"
        colors[numpy.argsort(temperature)[-1]] = "red"
        toyplot.plot(x, y, color=colors)
* Specify explicit per-datum color values for everything (i.e. map them yourself).
        toyplot.plot(x, y, color=mycolors)

**API**

* <strike>(values) - map the values with the default colormap.</strike>
* (colormap) - Use the given colormap to map series $[0, N)$.
    * What if there's only one series?  Users may expect it to map the datums.
* (palette) - Use the palette with a linear colormap to map series $[0, N)$.
    * What if there's only one series?  Users may expect it to map the datums.
* <strike>(values, colormap) - map values with the given colormap.</strike>
* <strike>(values, palette) - map values with a linear colormap and the given palette.</strike>

toyplot.color.broadcast(...) always returns a numpy color array (array with dtype == toyplot.color.dtype), with the given shape.

**Mark Colors**

* bars
    * Intrinsic: bar fill, bar stroke.
    * Controls: fill, style.
* fill
    * Intrinsic: region fill, region stroke.
    * Controls: fill, style.
* graph
    * Intrinsic: vertex marker fill, vertex marker stroke, vertex marker label fill, vertex marker label stroke, edge stroke.
    * Controls: vertex color, fill, edge color, edge style, mstyle, mlstyle.
* hlines
    * Intrinsic: line stroke.
    * Controls: stroke, style.
* matrix
    * Intrinsic: cell fill.
    * Controls: ???
* plot
    * Intrinsic: marker fill, marker stroke, marker label fill, marker label stroke, line stroke.
    * Controls: color, fill, style, mstyle, mlstyle.
* rect
    * Intrinsic: rectangle fill, rectangle stroke.
    * Controls: 
* scatterplot
    * Intrinsic: marker fill, marker stroke, marker label fill, marker label stroke.
    * Controls: color, fill, style, mstyle, mlstyle.
* text
    * Intrinsic: text fill, text stroke.
    * Controls: fill, style.
* vlines 
    * Intrinsic: line stroke.
    * Controls: stroke, style.

**Values**

* <strike>"value" - a single CSS color value.</strike>
* <strike>(r, g, b) - a single RGB color value.</strike>
* <strike>(r, g, b, a) - a single RGBA color value.</strike>
* <strike>toyplot.color.rgb() - a single RGB color value.</strike>
* <strike>toyplot.color.rgba() - a single RGBA color value.</strike>
* <strike>[value1, value2, ...] - 1D heterogeneous collection of CSS and tuple color values.</strike>
    * We don't allow scalar values in this case, because their domain would be ambiguous.
    * Have to be careful with the implementation here - we don't want a list of nothing-but-tuples to be misinterpreted as a 2D numpy array of scalars.
* <strike>numpy string array with shape $M$ - 1D collection of CSS color values.</strike>
* numpy string array with shape $M \times N$ - 2D collection of CSS color values.
* <strike>numpy numeric array with shape $M$ - 1D collection of scalar values for mapping.</strike>
* <strike>numpy numeric array with shape $M \times N$ - 2D collection of scalar values for mapping.</strike>
* <strike>numpy color array with shape $M$ - 1D collection of color values.</strike>
* numpy color array with shape $M \times N$ - 2D collection of color values.

In [1]:
import numpy
numpy.random.seed(1234)

x = numpy.linspace(0, 1)
y = x ** 2
temperature = numpy.random.uniform(size=y.shape)

In [2]:
import toyplot

In [19]:
toyplot.plot(x, y, size=200, marker="o", mstyle={}, color="red")

(<toyplot.canvas.Canvas at 0x10fe4c450>,
 <toyplot.mark.Plot at 0x10fe9d610>)

In [12]:
toyplot.bars(x, y, style={"fill":"black"}, fill="red")

(<toyplot.canvas.Canvas at 0x10fe9d710>,
 <toyplot.mark.BarMagnitudes at 0x10fe39d90>)